In [1]:
# Integration of biopsies and organ donors from 2 studies
import anndata, numpy as np, pandas as pd, imp, scanpy as sc, rpy2
import matplotlib.pyplot as plt
imp.reload(lpy)
%load_ext rpy2.ipython
%matplotlib inline
sc.logging.print_header()
pd.options.display.max_columns = None
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
import lpy # local lpy.py file

scanpy==1.9.1 anndata==0.7.8 umap==0.4.6 numpy==1.19.5 scipy==1.6.1 pandas==1.1.4 scikit-learn==0.22 statsmodels==0.12.2 python-igraph==0.8.3 louvain==0.7.0 leidenalg==0.8.3 pynndescent==0.5.2


In [2]:
adata = anndata.read_h5ad("int5.h5ad")

In [3]:
adata.obs["cohort"] = ["Base_" + x if y == "baseline" else ( y + "_" + z if x == "China" else y + "_CVID" ) for x,y,z in zip(adata.obs["StudyName"], adata.obs["Stage"], adata.obs["CovidSeverity"])]
freq = lpy.RTable(adata.obs, "general_integrated2", "cohort")
freq = freq.append(pd.DataFrame({x:y for x,y in zip(freq.columns,np.sum(freq.iloc[np.array([x < freq.shape[0]-1 for x in range(freq.shape[0])])],axis=0).tolist())},index=["Sum"]))
freq["base_Sum"] = freq["Base_Javi"] +  freq["Base_Haniffa"] + freq["Base_China"]
freq["prog_Sum"] = freq["progression_CVID"] +  freq["progression_Mild"] + freq["progression_Severe"]
freq["conv_Sum"] = freq["convalescence_CVID"] +  freq["convalescence_Mild"] + freq["convalescence_Severe"]

freq

,progression_Severe,progression_Mild,Base_Haniffa,Base_Javi,convalescence_Mild,Base_China,convalescence_CVID,progression_CVID,convalescence_Severe
T Naive CD4,921,634,10547,174,1027,12060,454,399,589
TCM CD4,1780,2634,8647,1349,1438,887,344,392,2385
TEM CD4,122,152,10805,81,294,364,105,149,263
T Effector CD4,344,304,883,1203,418,986,2556,1526,167
T Naive CD8,269,469,5043,100,448,1647,109,80,534
TEM CD8,1069,877,3923,922,908,3285,1202,851,548
TE CD8,1038,826,3791,1861,1473,2320,6647,3694,824
NKT-like,384,1118,2307,3194,1652,641,1849,3966,1588
NK CD16,3138,4782,10859,228,3164,5729,378,671,2185
NK CD56,69,139,2569,248,261,480,229,362,229


In [4]:
from scipy.stats import hypergeom as hg
import math
tab= pd.DataFrame(index=freq.index[range(len(freq.index)-2)],columns=["base_CVID_enrich","base_CVID_log10pvalue","prog_CVID_enrich","prog_CVID_log10pvalue","conv_CVID_enrich","conv_CVID_log10pvalue"])

sumindex = len(freq.index)-1;
for x in range(tab.shape[0]):
    
    tab["base_CVID_enrich"][x] = freq["Base_Javi"][x] * freq["base_Sum"][sumindex] / (freq["Base_Javi"][sumindex] * freq["base_Sum"][x])
    if tab["base_CVID_enrich"][x] >=1: # CVID enrichment
        tab["base_CVID_log10pvalue"][x] = hg.logcdf(freq["base_Sum"][x] - freq["Base_Javi"][x],freq["base_Sum"][sumindex],freq["base_Sum"][sumindex] - freq["base_Sum"][x],freq["Base_Javi"][sumindex]) /math.log(10) - math.log(0.5) / math.log(10)
    else:  # CVID depletion
        tab["base_CVID_log10pvalue"][x] = hg.logcdf(freq["Base_Javi"][x],freq["base_Sum"][sumindex],freq["base_Sum"][x],freq["Base_Javi"][sumindex]) - math.log(0.5) / math.log(10)
    
    tab["prog_CVID_enrich"][x] = freq["progression_CVID"][x] * freq["prog_Sum"][sumindex] / (freq["progression_CVID"][sumindex] * freq["prog_Sum"][x])
    if tab["prog_CVID_enrich"][x] >=1: # CVID enrichment
        tab["prog_CVID_log10pvalue"][x] = hg.logcdf(freq["prog_Sum"][x] - freq["progression_CVID"][x],freq["prog_Sum"][sumindex],freq["prog_Sum"][sumindex] - freq["prog_Sum"][x],freq["progression_CVID"][sumindex]) - math.log(0.5) / math.log(10)
    else:  # CVID depletion
        tab["prog_CVID_log10pvalue"][x] = hg.logcdf(freq["progression_CVID"][x],freq["prog_Sum"][sumindex],freq["prog_Sum"][x],freq["convalescence_CVID"][sumindex]) - math.log(0.5) / math.log(10)
    
    tab["conv_CVID_enrich"][x] = freq["convalescence_CVID"][x] * freq["conv_Sum"][sumindex] / (freq["convalescence_CVID"][sumindex] * freq["conv_Sum"][x])
    if tab["conv_CVID_enrich"][x] >=1: # CVID enrichment
        tab["conv_CVID_log10pvalue"][x] = hg.logcdf(freq["conv_Sum"][x] - freq["convalescence_CVID"][x],freq["conv_Sum"][sumindex],freq["conv_Sum"][sumindex] - freq["conv_Sum"][x],freq["convalescence_CVID"][sumindex])  - math.log(0.5) / math.log(10)
    else:  # CVID depletion
        tab["conv_CVID_log10pvalue"][x] = hg.logcdf(freq["convalescence_CVID"][x],freq["conv_Sum"][sumindex],freq["conv_Sum"][x],freq["convalescence_CVID"][sumindex]) - math.log(0.5) / math.log(10)
tab

Note that many logpvalue are incorrectly computed as -inf, as there are overflows in terms that would cancel out if computed differently, and the pvalue assumes that cell occur in sample independtly, which is far from the reality, so computing T test for frequency per donor instead:

In [5]:
adata.obs["SampleDonorID"] = [ y if x == "Other" else x for x,y in zip(adata.obs["CVID_Series"], adata.obs["demultiplexed"])]

freq = lpy.RTable(adata.obs, "general_integrated2", "SampleDonorID")
freq = freq.iloc[0:(freq.shape[0]-1)] 
del freq["doublet"]
del freq["unassigned"]
sums = np.sum(freq,axis=0).tolist()
for i in range(freq.shape[1]):
    freq[freq.columns[i]] = freq[freq.columns[i]] / sums[i]
freq

,BGCV02_CV0902,S-S059,S-M044,S-M046,P1 Conv,S-M047,S-S065,S-HC017,S-M051,P5 Base,S-HC015,P2 Conv,S-HC014,S-S061,MH8919333,S-S064,P5 Prog,S-M055,S-S062,P3 Conv,S-HC013,S-S055,P1 Base,MH8919227,BGCV14_CV0940,MH8919282,P3 Base,MH8919283,S-HC008,P4 Prog,BGCV10_CV0939,MH8919332,S-M067,MH8919177,P1 Prog,P2 Base,BGCV13_CV0934,S-S068,S-HC010,newcastle65,BGCV09_CV0917,P4 Base,BGCV08_CV0915,BGCV01_CV0902,P5 Conv,S-S057,BGCV05_CV0929,P3 Prog,BGCV04_CV0911,BGCV12_CV0926,S-S054,MH8919179,S-M052,S-S067,P4 Conv,newcastle74,P2 Prog,MH8919178,S-M050,MH8919226,BGCV15_CV0944,MH8919176,BGCV01_CV0904
T Naive CD4,0.063872,0.096947,0.061599,0.027160,0.006989,0.051018,0.032660,0.280610,0.041709,0.030441,0.242201,0.021418,0.217475,0.011400,0.108985,0.031088,0.009220,0.023475,0.023182,0.032007,0.381414,0.020041,0.011308,0.000000,0.049082,0.219919,0.011733,0.052189,0.167324,0.008425,0.199044,0.163482,0.018262,0.172088,0.003253,0.000600,0.107132,0.024182,0.080016,0.179873,0.140759,0.007897,0.155615,0.007626,0.002896,0.084692,0.125818,0.062530,0.049478,0.159483,0.010116,0.113411,0.032974,0.011741,0.049876,0.063968,0.014611,0.134449,0.006957,0.000000,0.070492,0.154935,0.059718
TCM CD4,0.208383,0.007499,0.111290,0.096790,0.001613,0.013823,0.174769,0.007828,0.043754,0.085743,0.002073,0.002295,0.003129,0.216601,0.037344,0.053408,0.072925,0.082929,0.022480,0.034400,0.002742,0.046830,0.023396,0.000000,0.116172,0.004676,0.240533,0.135068,0.070986,0.019938,0.204423,0.053597,0.113650,0.013116,0.003090,0.074063,0.261713,0.067466,0.096703,0.031913,0.241046,0.089339,0.206417,0.007626,0.133961,0.013587,0.091636,0.023901,0.188565,0.298981,0.157256,0.007726,0.084820,0.083104,0.004684,0.010194,0.021454,0.010457,0.085630,0.000000,0.175410,0.022021,0.308826
TEM CD4,0.104990,0.001607,0.018802,0.012593,0.002778,0.007288,0.012964,0.001405,0.003680,0.001522,0.000377,0.000765,0.000602,0.006413,0.057075,0.001594,0.025985,0.008421,0.000702,0.013461,0.001273,0.006339,0.011698,0.002180,0.127472,0.145418,0.001067,0.096744,0.037746,0.001123,0.058577,0.107706,0.000714,0.066107,0.002439,0.001799,0.101761,0.025808,0.039003,0.266727,0.231017,0.002468,0.160963,0.044805,0.014482,0.013813,0.181818,0.015693,0.112150,0.130878,0.001104,0.085817,0.008710,0.004770,0.001653,0.057616,0.006288,0.092172,0.014985,0.001892,0.140984,0.125049,0.243214
T Effector CD4,0.007186,0.033208,0.027218,0.031852,0.102858,0.019352,0.008227,0.015757,0.008996,0.070015,0.023089,0.090515,0.023589,0.013894,0.005674,0.004384,0.032691,0.014289,0.015104,0.260245,0.020074,0.014110,0.190680,0.000000,0.015537,0.011690,0.017067,0.013554,0.021690,0.012637,0.008966,0.015258,0.006019,0.011018,0.019027,0.008096,0.005670,0.003658,0.021311,0.021578,0.011103,0.006910,0.010695,0.005720,0.037654,0.016757,0.005818,0.195558,0.009896,0.007837,0.006989,0.006071,0.008917,0.005320,0.036098,0.005614,0.095247,0.004033,0.002676,0.000000,0.001639,0.008651,0.013340
T Naive CD8,0.117365,0.001607,0.046647,0.020247,0.001971,0.014325,0.052231,0.027098,0.011450,0.014206,0.040336,0.002040,0.007462,0.007481,0.055720,0.002790,0.004191,0.007910,0.008430,0.016153,0.026733,0.025153,0.006239,0.000000,0.012006,0.048784,0.017067,0.107493,0.117183,0.001404,0.017932,0.045775,0.004897,0.024659,0.001626,0.000900,0.135780,0.005487,0.039807,0.018314,0.013252,0.001234,0.055080,0.004766,0.013034,0.011096,0.003636,0.010381,0.101704,0.015674,0.012875,0.040287,0.021360,0.010457,0.001929,0.051854,0.003144,0.066776,0.005084,0.000000,0.021311,0.064884,0.087948
TEM CD8,0.041916,0.123728,0.034560,0.054568,0.038617,0.043981,0.014086,0.107186,0.028420,0.016743,0.065215,0.060938,0.068239,0.047025,0.012448,0.031088,0.008382,0.045165,0.033017,0.114568,0.062182,0.026994,0.121076,0.000000,0.034605,0.033198,0.089600,0.110453,0.048451,0.011514,0.017334,0.023080,0.029790,0.046695,0.009432,0.007496,0.041480,0.039220,0.030358,0.100635,0.031519,0.018509,0.035294,0.009533,0.023896,0.067255,0.048000,0.094399,0.015393,0.014890,0.008461,0.043874,0.021983,0.012842,0.031965,0.111981,0.064916,0.

In [6]:
import scipy.stats as stats
import math
cohortCVID = [["P1 Base","P2 Base","P3 Base","P4 Base","P5 Base"],
               ["P1 Prog","P2 Prog","P3 Prog","P4 Prog","P5 Prog"],
               ["P1 Conv","P2 Conv","P3 Conv","P4 Conv","P5 Conv"]]
cohortNCVID = [[x for x,y in adata[(adata.obs["Stage"] == "baseline")&(adata.obs["StudyName"] != "Javi")].obs["SampleDonorID"].value_counts().items()],
               [x for x,y in adata[(adata.obs["Stage"] == "progression")&(adata.obs["StudyName"] != "Javi")].obs["SampleDonorID"].value_counts().items()],
               [x for x,y in adata[(adata.obs["Stage"] == "convalescence")&(adata.obs["StudyName"] != "Javi")].obs["SampleDonorID"].value_counts().items()]]
tab= pd.DataFrame(index=freq.index[range(len(freq.index))],columns=["base_CVID_meanfreq","base_NCVID_meanfreq","base_CVID_log10pvalue","prog_CVID_meanfreq","prog_NCVID_meanfreq","prog_CVID_log10pvalue","conv_CVID_meanfreq","conv_NCVID_meanfreq","conv_CVID_log10pvalue"])

for x in range(tab.shape[0]):
    for y in range(3):
        cvidfreq =  [freq[i][x] for i in cohortCVID[y]]
        ncvidfreq = [freq[i][x] for i in cohortNCVID[y]]
        tab[tab.columns[y*3]][x] = np.mean(cvidfreq)
        tab[tab.columns[y*3+1]][x] = np.mean(ncvidfreq)
        tab[tab.columns[y*3+2]][x] = math.log(stats.ttest_ind(a=cvidfreq, b=ncvidfreq).pvalue) / math.log(10)
tab

,base_CVID_meanfreq,base_NCVID_meanfreq,base_CVID_log10pvalue,prog_CVID_meanfreq,prog_NCVID_meanfreq,prog_CVID_log10pvalue,conv_CVID_meanfreq,conv_NCVID_meanfreq,conv_CVID_log10pvalue
T Naive CD4,0.012396,0.130682,-2.31122,0.0196077,0.0423594,-0.6974,0.0226372,0.0289569,-0.301039
TCM CD4,0.102615,0.0939591,-0.0662667,0.0282617,0.0863696,-0.946648,0.0353906,0.080965,-1.00679
TEM CD4,0.00371079,0.0941179,-1.97764,0.0103057,0.00763032,-0.254465,0.0066278,0.0100437,-0.363641
T Effector CD4,0.0585537,0.0110685,-2.87895,0.0710319,0.0188222,-1.21877,0.105474,0.00987902,-2.33087
T Naive CD8,0.00792904,0.045654,-1.40089,0.00414941,0.0160365,-1.07614,0.0070254,0.0171846,-0.607657
TEM CD8,0.0506849,0.0435122,-0.175379,0.0377286,0.0484846,-0.225329,0.0539966,0.0337714,-0.769251
TE CD8,0.0897325,0.0399023,-1.26727,0.150299,0.050325,-1.4895,0.179719,0.0516826,-1.38473
NKT-like,0.177496,0.0210635,-7.67689,0.160615,0.0373264,-2.0789,0.0810901,0.0692058,-0.138802
NK CD16,0.0144658,0.129933,-2.22371,0.0325321,0.175742,-1.26965,0.0190922,0.124339,-2.28276
NK CD56,0.0172556,0.0288437,-0.447467,0.0156782,0.00549127,-1.10773,0.0119953,0.010573,-0.17912
